In [5]:
!pip install captum torchtext==0.18.0 torch==2.3.1 datasets

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from datasets import load_dataset
import re
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data and Dictionary
dataset = load_dataset("imdb")
train_data = dataset['train'].shuffle(seed=42) # Only 1000 data for speed up but now wole dataset

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.split()

vocab = build_vocab_from_iterator([clean_text(x['text']) for x in train_data], min_freq=2, specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Basic LSTM Model
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        # forward function must be simpler for Captum
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden[-1])

model = LSTMClassifier(len(vocab), 100, 64, 2).to(device) # 2 Outputs: Negative, Pozitive

# Fast Training
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

model.train()
for i, item in enumerate(train_data):
    # if i > 100: break # 100 steps is enough
    text = torch.tensor([vocab[t] for t in clean_text(item['text'])]).to(device).unsqueeze(0)
    label = torch.tensor([item['label']]).to(device)

    optimizer.zero_grad()
    preds = model(text)
    loss = criterion(preds, label)
    loss.backward()
    optimizer.step()

In [7]:
from captum.attr import IntegratedGradients, LayerIntegratedGradients
from captum.attr import visualization as viz

model.train()

# Example sentence to analyze
ornek_yorum = "this movie was absolutely wonderful and brilliant"
label = 1 # 1: Pozitive

# Prepare sentence for model
tokens = clean_text(ornek_yorum)
input_indices = torch.tensor([vocab[t] for t in tokens]).to(device).unsqueeze(0)

# Define Integrated Gradients
lig = LayerIntegratedGradients(model, model.embedding)

# Calculate (Target: label=1 meaning Pozitive)
attributions, delta = lig.attribute(input_indices, target=label, n_steps=50, return_convergence_delta=True)


# Visualization
def visualize_attributions(attributions, tokens):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    print("\n--- Word Scores ---")
    print("(Pozitive scores support decision, negative ones hinder)\n")

    for token, score in zip(tokens, attributions):
        # A basic bar for visualization
        bar_len = int(abs(score) * 50)
        bar = "█" * bar_len
        if score > 0:
            print(f"{token:15} | {score:+.4f} | \033[92m{bar} (POSITIVE)\033[0m") # Green
        else:
            print(f"{token:15} | {score:+.4f} | \033[91m{bar} (NEGATIVE)\033[0m") # Red

visualize_attributions(attributions, tokens)


--- Word Scores ---
(Pozitive scores support decision, negative ones hinder)

this            | +0.0148 |  (POSITIVE)
movie           | -0.1885 | █████████ (NEGATIVE)
was             | +0.6033 | ██████████████████████████████ (POSITIVE)
absolutely      | -0.4452 | ██████████████████████ (NEGATIVE)
wonderful       | +0.0230 | █ (POSITIVE)
and             | -0.1221 | ██████ (NEGATIVE)
brilliant       | +0.6217 | ███████████████████████████████ (POSITIVE)


In [8]:
import torch.nn.functional as F

def analyze_uncertainty(text):
    tokens = clean_text(text)
    input_indices = torch.tensor([vocab[t] for t in tokens]).to(device).unsqueeze(0)

    with torch.no_grad():
        logits = model(input_indices)
        probs = F.softmax(logits, dim=1)

        # Entropi Calculation: -sum(p * log(p))
        entropy = -torch.sum(probs * torch.log(probs + 1e-9), dim=1).item()

        pred_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_class].item()

    return pred_class, confidence, entropy

# Error scenario (an example sentence)
# "Movie wasn't bad" -> It actually means good but since it contains the word "bad" model could suprised
diff_sentence = "the movie was not bad actually"

pred, conf, entropy = analyze_uncertainty(diff_sentence)

print(f"Sentence: '{diff_sentence}'")
print(f"Prediction: {'Pozitive' if pred == 1 else 'Negative'}")
print(f"Confidence: {conf:.4f}")
print(f"Entropy: {entropy:.4f} (If it is high model is indecisive)")

Sentence: 'the movie was not bad actually'
Prediction: Pozitive
Confidence: 0.5415
Entropy: 0.6897 (If it is high model is indecisive)
